# attention based seq2seq 로 summarization 구현
이상헌 <br>
voithru<br>

## 0. dataset 구현

마땅히 알맞은 데이터 셋을 찾지 못하다가 <br>

딥마인드의 [해당 논문](https://arxiv.org/abs/1506.03340) 에서 사용한 데이터셋은 **DMQA** 이다. <br>

여기서는 QnA의 모델링으로 사용하였는데, 해당 파일 안에 summary도 있나보다. <br>

### 0.1. 다운로드 및 밑작업
일단 다운로드를 시작한다. <br>
다음과 같은 [위치](http://cs.nyu.edu/~kcho/DMQA//)에서 받는다. <br>
```
CNN_stories.tgz
dailymail_stories.tgz
```
now unzip it <br>

이 안에는 @highlight로 emphasis 되어있는데, 이게 summary 라고 하는 것이다. <br>

밑작업을 위해서는 해당 데이터를 **tokenize**, 그리고 **binary** 로 만들어야 할 필요가 있다. <br>

cf) <br>
tokenize는 무엇인가. <br>
![tokenize 설명 img](https://cloud.githubusercontent.com/assets/2272790/18410099/1d0a1c1a-7761-11e6-9fe1-bd2e5622b90a.png)
* GO - ```<start>``` token. decoder에 가장 첫 번째 node에 들어갈 token.
* EOS - ```<end>``` token.
* UNK - unkown token. vocab에 들어있지 않는 rare vocab을 replace하기 위한 token. 우리 seq2seq에서는 사용되지 않는다! 하지만 굉장히 더러운 data에 있어서는 사용해야 한다. 예) ```my name is skdy33``` => ```my name is _unk_```
* PAD - mini-batch 안에 들어있는 데이터는 같은 길이를 가져야 한다. 따라서 짧은 데이터는 ```_pad_``` 토큰이 뒤에 붙는다.



### 참고자료
* [google's text summarization code](https://github.com/pranay360/TextSum_Data_Generation)
* [data-generation code](https://github.com/pranay360/TextSum_Data_Generation)
* [Get to the point : summarization with  pointer-generator networks](https://arxiv.org/pdf/1704.04368.pdf)
* [what does PAD / GO / EOS / UNK mean?](https://github.com/nicolas-ivanov/tf_seq2seq_chatbot/issues/15)
* [utf8 and encoding](wholetext=re.sub(r'[^\x00-\x7F]+','', wholetext)

In [3]:
# data conversion 시작

# 가장 먼저 nltk 설치하고 오세요
# sudo apt-get install python-nltk
# pip install nltk

import os
import re
import sys
import struct
import numpy as np
import collections
from nltk.tokenize import sent_tokenize


In [28]:
# word dictionary를 위한 counter 객체
counter = collections.Counter()
temp=0
#train, test, validation split
tr_r=0.85
v_r=0.08
# directory list
files = os.listdir('data/cnn/stories/')
# file 수
n_files = len(files)
print(n_files)
# train
train=files[:int(n_files*0.8)]
validation=files[len(train):len(train)+int(n_files*0.12)]
test=files[len(train)+len(validation):]

92579


In [40]:
with open("data/cnn/stories/" + files[0],'r') as f:
    # decode 맞춰야 하나?
    txt = f.read().lower()
    # delete non-ascii
    txt = re.sub(r'[^\x00-\x7F]+','',txt)
    # '안녕' 을 ' 안녕 ' 으로 만든다.
    txt=re.sub(r"(\s?[\']\s+|\s+[\']\s?)"," ' ", txt)
    # "안녕" 을 " 안녕 " 으로 만든다.
    txt=re.sub(r'(\s?[\"]\s+|\s+[\"]\s?)',' " ', txt)
    print(txt)

(cnn) -- the first anniversary of the egyptian revolution is today. egyptian society and the forces in egypt are in a state of anticipation. world media has its cameras and correspondents in cairo and major cities around the country. but many egyptians wonder if the revolution amounted to nothing more than a military coup.

it has been a year since the eruption of the first egyptian revolution that stunned the world and ended 30 years of authoritarian, oppressive and corrupt rule by hosni mubarak, egypt's last pharaoh. mubarak ended up in jail along with his sons and his regime's major figures, with stories of their unimaginable corruption, brutality and looting surfacing ever since.

but since then, most egyptians have become angry and frustrated with the performance of the military council -- comprising more than a dozen elderly generals -- that has taken control. egyptians, political forces and revolutionaries accuse the military council of being accomplices with the remnants of the

In [46]:
for i,fi in enumerate(files):
    with open('data/cnn/stories/'+fi,'r') as f:
        txt = f.read()
        print(re.findall(r"(\s?[\']\s+|\s+[\']\s?)",txt))
        print(txt)
        if(i == 20):
            break

[]
(CNN) -- The first anniversary of the Egyptian revolution is today. Egyptian society and the forces in Egypt are in a state of anticipation. World media has its cameras and correspondents in Cairo and major cities around the country. But many Egyptians wonder if the revolution amounted to nothing more than a military coup.

It has been a year since the eruption of the first Egyptian revolution that stunned the world and ended 30 years of authoritarian, oppressive and corrupt rule by Hosni Mubarak, Egypt's last pharaoh. Mubarak ended up in jail along with his sons and his regime's major figures, with stories of their unimaginable corruption, brutality and looting surfacing ever since.

But since then, most Egyptians have become angry and frustrated with the performance of the Military Council -- comprising more than a dozen elderly generals -- that has taken control. Egyptians, political forces and revolutionaries accuse the Military Council of being accomplices with the remnants of 

"(cnn) -- the first anniversary of the egyptian revolution is today. egyptian society and the forces in egypt are in a state of anticipation. world media has its cameras and correspondents in cairo and major cities around the country. but many egyptians wonder if the revolution amounted to nothing more than a military coup.\n\nit has been a year since the eruption of the first egyptian revolution that stunned the world and ended 30 years of authoritarian, oppressive and corrupt rule by hosni mubarak, egypt's last pharaoh. mubarak ended up in jail along with his sons and his regime's major figures, with stories of their unimaginable corruption, brutality and looting surfacing ever since.\n\nbut since then, most egyptians have become angry and frustrated with the performance of the military council -- comprising more than a dozen elderly generals -- that has taken control. egyptians, political forces and revolutionaries accuse the military council of being accomplices with the remnants o

In [29]:
files[0]


'a12c107266c4d55a0b2acdcdf98ec1c70bef9417.story'